# Structured Q&A

Source code: https://github.com/mozilla-ai/structured-qa

Docs: https://mozilla-ai.github.io/structured-qa

## Installing dependencies

In [1]:
!git clone --single-branch --branch 5-add-benchmark https://github.com/mozilla-ai/structured-qa

Cloning into 'structured-qa'...
remote: Enumerating objects: 939, done.
remote: Counting objects: 100% (377/377), done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 939 (delta 246), reused 220 (delta 140), pack-reused 562 (from 1)
Receiving objects: 100% (939/939), 2.56 MiB | 9.80 MiB/s, done.
Resolving deltas: 100% (528/528), done.


In [2]:
%pip install ./structured-qa

Processing ./structured-qa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00
  Created wheel for structured-qa: filename=structured_qa-0.3.3.dev113+g0ab4688-py3-none-any.whl size=13247 sha256=331a7e775095111e06c2431f076e99cfda44a105ced2485b20b2264

In [3]:
%pip install --quiet https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.4-cu122/llama_cpp_python-0.3.4-cp311-cp311-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.2/445.2 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00


# Setup

In [4]:
import os

os.environ["LOGURU_LEVEL"] = "INFO"

In [5]:
from loguru import logger

## Function to Process all questions for a single Section

In [6]:
ANSWER_WITH_TYPE_PROMPT = """
You are a rigorous assistant answering questions.
You must only answer based on the current information available which is:

```
{CURRENT_INFO}
```

If the current information available not enough to answer the question,
you must return "I need more info" srting and nothing else:

If the current information is enough to answer, you must return one of the following formats:
- YES/NO (for boolean questions)
- Number (for numeric questions)
- Single letter (for multiple-choice questions)
"""


def process_section_questions(
    section_file,
    section_data,
    model,
):
    logger.info("Predicting")
    answers = {}
    sections = {}
    for index, row in section_data.iterrows():
        question = row["question"]
        logger.info(f"Question: {question}")
        messages = [
            {
                "role": "system",
                "content": ANSWER_WITH_TYPE_PROMPT.format(
                    CURRENT_INFO=section_file.read_text()
                ),
            },
            {"role": "user", "content": question},
        ]
        response = model.get_response(messages)
        logger.info(f"Answer: {response}")
        answers[index] = response
        sections[index] = None
    return answers, sections

## Load Model

In [7]:
from structured_qa.model_loaders import load_llama_cpp_model

In [8]:
model = load_llama_cpp_model(
    "bartowski/Qwen2.5-7B-Instruct-GGUF/Qwen2.5-7B-Instruct-Q8_0.gguf"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Qwen2.5-7B-Instruct-Q8_0.gguf:   0%|          | 0.00/8.10G [00:00<?, ?B/s]

# Run Benchmark

In [9]:
from pathlib import Path

import pandas as pd


logger.info("Loading input data")
data = pd.read_csv("structured-qa/benchmark/structured_qa.csv")
data["pred_answer"] = [None] * len(data)
data["pred_section"] = [None] * len(data)

for section_name, section_data in data.groupby("section"):
    section_file = Path(f"structured-qa/benchmark/perfect_context/{section_name}.txt")

    answers, sections = process_section_questions(section_file, section_data, model)

    for index in section_data.index:
        data.loc[index, "pred_answer"] = str(answers[index]).upper()
        data.loc[index, "pred_section"] = sections[index]

data.to_csv("results.csv")

2025-02-04 18:01:42.914 | INFO     | __main__:<cell line: 0>:6 - Loading input data
2025-02-04 18:01:42.952 | INFO     | __main__:process_section_questions:24 - Predicting
2025-02-04 18:01:42.956 | INFO     | __main__:process_section_questions:29 - Question: In billions, how many trainable parameters does GPT-3 have?
2025-02-04 18:01:44.761 | INFO     | __main__:process_section_questions:40 - Answer: 175
2025-02-04 18:01:44.763 | INFO     | __main__:process_section_questions:29 - Question: Does LoRA introduce additional inference latency compared to full fine-tuning?
2025-02-04 18:01:44.882 | INFO     | __main__:process_section_questions:40 - Answer: NO
2025-02-04 18:01:44.891 | INFO     | __main__:process_section_questions:24 - Predicting
2025-02-04 18:01:44.894 | INFO     | __main__:process_section_questions:29 - Question: What fire resistance must vertical partitions have? -A: EI30 -B: EI60 -C: EI90
2025-02-04 18:01:45.468 | INFO     | __main__:process_section_questions:40 - Answer:

# Results

In [10]:
results = pd.read_csv("results.csv")
for index, result in results.iterrows():
    if result["pred_answer"].startswith(
        (f"-{result['answer']}", f"{result['answer']}")
    ):
        results.loc[index, "pred_answer"] = result["answer"]
results.loc[results["answer"] != results["pred_answer"]]

,Unnamed: 0,document,type,section,question,answer,pred_answer,pred_section
10,10,https://arxiv.org/pdf/1706.03762,Scientific Paper,5.4 Regularization,What was the dropout rate used for the base mo...,0.1,PDROP = 0.1,NaN
28,28,https://arxiv.org/pdf/2201.11903,Scientific Report,3.1 Experimental Setup,How many large language models were evaluated?,5,FIVE,NaN
32,32,https://arxiv.org/pdf/2201.11903,Scientific Report,5 Symbolic Reasoning,Which symbolic reasoning task is used as an ou...,A,I NEED MORE INFO,NaN
37,37,https://github.com/mozilla-ai/structured-qa/re...,Board Game,CARD AND TILE EFFECTS,How many different races are there?,6,5,NaN
41,41,https://github.com/mozilla-ai/structured-qa/re...,Board Game,CHAPTER OVERVIEW,"After taking a landmark tile, do you reveal a ...",NO,YES,NaN
42,42,https://github.com/mozilla-ai/structured-qa/re...,Board Game,CARD AND TILE COSTS,Can a player pay coins to compensate for missi...,YES,NO,NaN
55,55,https://github.com/mozilla-ai/structured-qa/re...,Board Game,EXPEDITION PHASE,Do you need a fish to conquer a distant island?,YES,NO,NaN
58,58,https://github.com/mozilla-ai/structured-qa/re...,Board Game,LOCATION ABILITIES,How many victory points are granted by a built...,1,I NEED MORE INFO,NaN
68,68,https://docs.nvidia.com/cuda/pdf/CUDA_C_Progra...,Techincal Documentation,5.2. Thread Hierarchy,Can you identify a thread with a four-dimensio...,NO,I NEED MORE INFO,NaN
94,94,https://arxiv.org/pdf/2302.13971,Scientific Report,3 Main results,Was the model compared against GPT-4?,NO,I NEED MORE INFO,NaN


In [11]:
accuracy = sum(results["answer"] == results["pred_answer"]) / len(results)
accuracy

0.9029126213592233